In [1]:
import numpy as np
import torch
import tifffile
from pathlib import Path
import time
import os
import torch.nn as nn
import torch.optim as optim

PROJECT_ROOT = Path("/mnt/home/dchhantyal/3d-cnn-classification")
import sys
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
sys.path.append(str(PROJECT_ROOT))

/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <

In [2]:
from model import ConvRNN, Config, resize_volume, NucleusDataset, DataLoader, RandomAugmentation3D
from predict import predict_single_sample


In [3]:
print("\n--- Running Prediction Example ---")

# Create Config instance to access attributes
config = Config()

# Look for model file in common locations
possible_model_paths = [
    "nucleus_classifier_model.pth",
    "model/nucleus_classifier_model.pth",
    str(PROJECT_ROOT / "model/notebooks" / "final_model.pth"),
    str(PROJECT_ROOT / "model" / "nucleus_classifier_model.pth"),
    str(PROJECT_ROOT / "checkpoints" / "nucleus_classifier_model.pth"),
    str(PROJECT_ROOT / "saved_models" / "nucleus_classifier_model.pth"),
]

model_path = None
for path in possible_model_paths:
    if os.path.exists(path):
        model_path = path
        break

if model_path is None:
    print("❌ No trained model found!")
    print("\nLooked for model file in:")
    for path in possible_model_paths:
        print(f"  - {path}")
    print("\n📝 To train a model first, run:")
    print("  python model/train.py")
    print("\n📝 Or place your trained model file in one of the above locations.")
    exit(1)

# 1. Initialize a new model instance and load the saved weights
try:
    inference_model = ConvRNN(num_classes=config.num_classes).to(config.device)
    inference_model.load_state_dict(
        torch.load(model_path, map_location=config.device)
    )
    print(f"✅ Trained model loaded from: {model_path}")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit(1)



--- Running Prediction Example ---
✅ Trained model loaded from: /mnt/home/dchhantyal/3d-cnn-classification/model/notebooks/final_model.pth


In [4]:
# 2. Define the file paths for your new, unseen sample
new_sample_files = [
    str(
        PROJECT_ROOT
        / "data"
        / "nuclei_state_dataset"
        / "mitotic"
        / "221016FUCCINanogstack3_nucleus1_frame19_count9"
        / "previous"
        / "raw_original.tif"
    ),
    str(
        PROJECT_ROOT
        / "data"
        / "nuclei_state_dataset"
        / "mitotic"
        / "221016FUCCINanogstack3_nucleus1_frame19_count9"
        / "current"
        / "raw_original.tif"
    ),
    str(
        PROJECT_ROOT
        / "data"
        /"nuclei_state_dataset"
        / "mitotic"
        / "221016FUCCINanogstack3_nucleus1_frame19_count9"
        / "next"
        / "raw_original.tif"
    ),
]

# Check if sample files exist
missing_files = [f for f in new_sample_files if not os.path.exists(f)]
if missing_files:
    print("\n⚠️  Sample files not found:")
    for f in missing_files:
        print(f"  - {f}")
    print("\nUsing dummy prediction instead...")

    # Create a dummy prediction for demonstration
    print(f"\nDummy Prediction Results:")
    print(f"Predicted Class: mitotic")
    print(f"Confidence: 0.8547")
    exit(0)

# 3. Run the prediction
try:
    predicted_label, confidence_score = predict_single_sample(
        model=inference_model, file_paths=new_sample_files, device=config.device
    )
    print(f"\nPrediction Results:")
    print(f"Predicted Class: {predicted_label}")
    print(f"Confidence: {confidence_score:.4f}")
except Exception as e:
    print(f"\n❌ Error during prediction: {e}")
    print("\nThis could be due to:")
    print("  - Incompatible model architecture")
    print("  - Corrupted model file")
    print("  - Invalid input data format")



Prediction Results:
Predicted Class: new_daughter
Confidence: 0.8866
